In [1]:
sc

In [2]:
df = spark.read.load('wunderground_2011-2013.csv', format='csv', header=True, inferSchema=False)

In [3]:
df.dtypes

[('TimeEDT', 'string'),
 ('TemperatureF', 'string'),
 ('Dew PointF', 'string'),
 ('Humidity', 'string'),
 ('Sea Level PressureIn', 'string'),
 ('VisibilityMPH', 'string'),
 ('Wind Direction', 'string'),
 ('Wind SpeedMPH', 'string'),
 ('Gust SpeedMPH', 'string'),
 ('PrecipitationIn', 'string'),
 ('Events', 'string'),
 ('Conditions', 'string'),
 ('WindDirDegrees', 'string'),
 ('DateUTC', 'string')]

In [4]:
from pyspark.sql import Row
import csv
from dateutil.parser import parse
from dateutil.tz import gettz
from dateutil import tz
# from datetime import datetime


from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')


def process_wunder(idx, part):
    if idx == 0:
        part.next()
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    for p in csv.reader(part):
        # deal with edge case of 'No daily or hourly history data available'
        # for 1/28/2012, 1/29/2012, 1/30/2012
        if len(p) < 12:
            continue
        freezing = '0'
        windy = '0'
        fog = '0'
        rain = '0'
        snow = '0'
        if  float(p[1]) <= 32:
            freezing = '1'
        try: 
            if float(p[8]) >= 30:
                windy = '1'
        except:
            pass
        try:
            if  0 <= float(p[5]) <= 2 or 'Fog' in p[10] or 'Mist' in p[11] :
                fog = '1'
        except: 
            pass

        if 'Rain' in p[10] or 'Rain' in p[11] or 'Rain' in p[11] or 'Thunderstorm' in p[11]:
            rain = '1'

        if 'Snow' in p[10] or 'Snow' in p[11] or 'Snow' in p[10]:
            snow = '1'
        
        DateUTC = parse(p[13])
        DatetimeEDT = DateUTC.replace(tzinfo=from_zone).astimezone(to_zone)
        
        yield Row(  
                    DateUTC = p[13],
                    TimeEDT = p[0],
                    TemperatureF = p[1],
                    Dew_PointF = p[2],
                    Humidity = p[3],
                    Sea_Level_PressureIn = p[4],
                    VisibilityMPH = p[5],
                    Wind_Direction = p[6],
                    Wind_SpeedMPH = p[7],
                    Gust_SpeedMPH = p[8],
                    PrecipitationIn = p[9],
                    Events = p[10],
                    Conditions = p[11],
                    WindDirDegrees = p[12],
                    Freezing = freezing,
                    Windy = windy,
                    LowVisibility = fog,
                    Rain = rain,
                    Snow = snow,
                    DatetimeEDT = DatetimeEDT,
                    )
        
rows = sc.textFile('wunderground_2011-2017_final.csv').mapPartitionsWithIndex(process_wunder)
df = sqlContext.createDataFrame(rows)
df.toPandas().to_csv('processed_2011-2017_fixed.csv')
# df.coalesce(1).write.csv('processed_2011-2017_fixed.csv', header=True)

KeyboardInterrupt: 

In [27]:
# df.dtypes
# df.select("DateUTC", "Snow", "Windy", "Rain").toPandas().to_csv('test2.csv')
df.select("DateUTC", "DatetimeEDT", "TimeEDT").toPandas().to_csv('test.csv')


if temp < 32: freezeing bool true
wind speed > 31 mph: windy true
0< visibilty < 1 mile: low visiiablity
rain: rain 
snow: snow


In [106]:
# df.toPandas().to_csv('processed_2011-2013.csv')

In [23]:
from dateutil.parser import parse
from dateutil.tz import gettz
from datetime import datetime
from dateutil import tz

utc_datetime = '2011-01-02 06:48:00'
edt_datetime = '1:48 AM'
edt_datetime = '2011-01-02 1:48:00'

utc = parse(utc_datetime)

from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')

# Tell the datetime object that it's in UTC time zone since 
# datetime objects are 'naive' by default
utc = utc.replace(tzinfo=from_zone)

# # Convert time zone
est = utc.astimezone(to_zone)

print(utc)
print(est)

2011-01-02 06:48:00+00:00
2011-01-02 01:48:00-05:00


In [39]:
# df.take(2)
unique_Conditions = df.select("Conditions").distinct()
unique_Conditions.count()

18

In [41]:
unique_Conditions.show()

+-------------------+
|         Conditions|
+-------------------+
|       Thunderstorm|
|Light Freezing Rain|
|   Scattered Clouds|
|                Fog|
|              Clear|
|      Partly Cloudy|
| Light Freezing Fog|
|            Unknown|
|      Mostly Cloudy|
|         Heavy Rain|
|         Light Rain|
|               Mist|
|               Snow|
|               Rain|
|               Haze|
|           Overcast|
|         Light Snow|
|         Heavy Snow|
+-------------------+



In [43]:
unique_Events = df.select("Events").distinct()
print unique_Events.count()
unique_Events.show()

7
+------------+
|      Events|
+------------+
|Thunderstorm|
|    Fog-Snow|
|         Fog|
|    Fog-Rain|
|        Rain|
|        Snow|
|            |
+------------+



In [51]:
linkinfo = spark.read.load('linkinfo.csv', format='csv', header=True, inferSchema=False)
linkinfo.dtypes

[('linkId', 'string'),
 ('linkPoints', 'string'),
 ('EncodedPolyLine', 'string'),
 ('EncodedPolyLineLvls', 'string'),
 ('Transcom_id', 'string'),
 ('Borough', 'string'),
 ('linkName', 'string'),
 ('Owner', 'string')]

In [52]:
april2015 = spark.read.load('april2015.csv', format='csv', header=True, inferSchema=False)
april2015.dtypes
# april2015.rdd.take(5)

[('Id', 'string'),
 ('Speed', 'string'),
 ('TravelTime', 'string'),
 ('Status', 'string'),
 ('DataAsOf', 'string'),
 ('linkId', 'string')]

In [61]:

def map_linkID(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        yield (p[5], (p[0], p[1], p[2], p[3], p[4]) )

april2015_rdd = sc.textFile('april2015.csv').mapPartitionsWithIndex(map_linkID)
april2015_rdd.take(5)

[('4616337', ('1', '16.78', '303', '0', '4/18/2015 09:05:34')),
 ('4616325', ('2', '21.75', '145', '0', '4/18/2015 09:05:34')),
 ('4616324', ('3', '16.78', '358', '0', '4/18/2015 09:05:34')),
 ('4616338', ('4', '21.13', '154', '0', '4/18/2015 09:05:34')),
 ('4616323', ('106', '16.78', '131', '0', '4/18/2015 09:05:34'))]

In [63]:
def map_linkinfo_id(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        yield (p[0], (p[1], p[2], p[3], p[4], p[5], p[6],7) )

linkinfo_rdd = sc.textFile('linkinfo.csv').mapPartitionsWithIndex(map_linkinfo_id)
linkinfo_rdd.take(2)

[('4616337',
  ('40.74047,-74.009251 40.74137,-74.00893 40.7431706,-74.008591 40.7462304,-74.00797 40.74812,-74.007651 40.748701,-74.007691 40.74971,-74.00819 40.75048,-74.008321 40.751611,-74.00789 40.7537504,-74.00704 40.75721,-74.00463 40.76003,-74.002631 40.7607405,-7',
   '}btwFx|ubMsD_AgJcAcR{ByJ_AsBFiEbByCXaFuAkLiDsTaNsPoKmCmB',
   'BBBBBBBBBBBBB',
   '4616337',
   'Manhattan',
   '11th ave n ganservoort - 12th ave @ 40th st',
   7)),
 ('4616325',
  ('40.73933,-74.01004 40.73895,-74.01012 40.7376,-74.010021 40.7346,-74.01026 40.72912,-74.010781 40.72619,-74.011131',
   'y{swFvavbMjANlGSvQn@fa@fBhQdA',
   'BBBBBB',
   '4616325',
   'Manhattan',
   '11th ave s ganservoort - west st @ spring st',
   7))]

In [65]:
april2015_rdd.join(linkinfo_rdd).take(2)

[('4616297',
  (('178', '52.2', '162', '0', '4/18/2015 09:05:34'),
   ('40.8302204,-73.850641 40.8295305,-73.848401 40.8291004,-73.847491 40.82836,-73.8459 40.8281405,-73.84524 40.8280305,-73.844641 40.827991,-73.843911 40.82804,-73.842981 40.8281906,-73.841761 40.8286304,-73.83844 40.82887,-73.83673 40.8291904,-73.83548 40.8',
    '{sexFn}vaMhC_MtAuDrC}Hj@cCTwBFqCIyD]sFwAwSo@uI_AyFe@wBmAmE}A}DwAsCyFqK{DmH}AuD{D_KeCiCiCmAuDQwC`@kH`B',
    'BBBBBBBBBBBBBBBBBBBBBBBBB',
    '4616297',
    'Bronx',
    'CBE E CASTLE HILL AVE - BE N WATERBURY AVE',
    7))),
 ('4616297',
  (('178', '50.95', '166', '0', '4/18/2015 09:10:34'),
   ('40.8302204,-73.850641 40.8295305,-73.848401 40.8291004,-73.847491 40.82836,-73.8459 40.8281405,-73.84524 40.8280305,-73.844641 40.827991,-73.843911 40.82804,-73.842981 40.8281906,-73.841761 40.8286304,-73.83844 40.82887,-73.83673 40.8291904,-73.83548 40.8',
    '{sexFn}vaMhC_MtAuDrC}Hj@cCTwBFqCIyD]sFwAwSo@uI_AyFe@wBmAmE}A}DwAsCyFqK{DmH}AuD{D_KeCiCiCmAuDQwC`@kH`B',
